In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import skl2onnx as sk

print("Its alive!")

ModuleNotFoundError: No module named 'skl2onnx'

In [4]:
#Functions
def fit_crossvalidate_reg(df, label, k=10, r=5, repeat=True):
  import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  from numpy import mean, std
  from sklearn import svm
  from sklearn import gaussian_process
  from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
  from xgboost import XGBRegressor

  X = df.drop(columns=[label])
  y = df[label]

  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
  else:
    cv = KFold(n_splits=k, random_state=12345, shuffle=True)
  
  fit = {}    # Use this to store each of the fit metrics
  models = {} # Use this to store each of the models

  # Create the model objects
  model_ols = lm.LinearRegression()
  model_rr = lm.Ridge(alpha=0.5) # adjust this alpha parameter for better results (between 0 and 1)
  model_lr = lm.Lasso(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
  model_llr = lm.LassoLars(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
  model_br = lm.BayesianRidge()
  model_pr = lm.TweedieRegressor(power=1, link="log") # Power=1 means this is a Poisson
  model_gr = lm.TweedieRegressor(power=2, link="log") # Power=2 means this is a Gamma
  model_igr = lm.TweedieRegressor(power=3) # Power=3 means this is an inverse Gamma
  model_svm = svm.SVR()
  model_lsvm = svm.LinearSVR()
  model_nusvm = svm.NuSVR()
  model_gpr = gaussian_process.GaussianProcessRegressor(DotProduct() + WhiteKernel())
  model_df = se.RandomForestRegressor(random_state=12345)
  model_etr = se.ExtraTreesRegressor(random_state=12345)
  model_abr = se.AdaBoostRegressor(n_estimators=100, random_state=12345)
  model_gbr = se.GradientBoostingRegressor(random_state=12345)
  model_hgbr = se.HistGradientBoostingRegressor(random_state=12345)
  model_vr = se.VotingRegressor(estimators=[('DF', model_df), ('ETR', model_etr), ('ABR', model_abr), ('GBR', model_gbr)])
  estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=42)), ('svr', svm.SVR(C=1, gamma=1e-6))]
  model_sr = se.StackingRegressor(estimators=estimators, final_estimator=se.GradientBoostingRegressor(random_state=12345))
  model_xgb = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
  
  # Fit a crss-validated R squared score and add it to the dict
  fit['OLS'] = mean(cross_val_score(model_ols, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Ridge'] = mean(cross_val_score(model_rr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Lasso'] = mean(cross_val_score(model_lr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['LARS'] = mean(cross_val_score(model_llr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Bayesian'] = mean(cross_val_score(model_br, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Poisson'] = mean(cross_val_score(model_pr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Gamma'] = mean(cross_val_score(model_gr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Inverse'] = mean(cross_val_score(model_igr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['SupportVM'] = mean(cross_val_score(model_svm, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Linear SVM'] = mean(cross_val_score(model_lsvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['NuSupportVM'] = mean(cross_val_score(model_nusvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['GaussianP'] = mean(cross_val_score(model_gpr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Dec Forest'] = mean(cross_val_score(model_df, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Extra Trees'] = mean(cross_val_score(model_etr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['AdaBoost DT'] = mean(cross_val_score(model_abr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Grad. Boost'] = mean(cross_val_score(model_gbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['HG Boost'] = mean(cross_val_score(model_hgbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Voting'] = mean(cross_val_score(model_vr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Stacking'] = mean(cross_val_score(model_sr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='r2', cv=cv, n_jobs=-1))

  # Add the model to another dict; make sure the keys have the same names as the list above
  models['OLS'] = model_ols
  models['Ridge'] = model_rr
  models['Lasso'] = model_lr
  models['LARS'] = model_llr
  models['Bayesian'] = model_br
  models['Poisson'] = model_pr
  models['Gamma'] = model_gr
  models['Inverse'] = model_igr
  models['SupportVM'] = model_svm
  models['Linear SVM'] = model_lsvm
  models['NuSupportVM'] = model_nusvm
  models['GaussianP'] = model_gpr
  models['Dec Forest'] = model_df
  models['Extra Trees'] = model_etr
  models['AdaBoost DT'] = model_abr
  models['Grad. Boost'] = model_gbr
  models['HG Boost'] = model_hgbr
  models['Voting'] = model_vr
  models['Stacking'] = model_sr
  models['XGBoost'] = model_xgb

  # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
  df_fit = pd.DataFrame({'R-squared':fit})
  df_fit.sort_values(by=['R-squared'], ascending=False, inplace=True)
  best_model = df_fit.index[0]
  print(df_fit)

  return models[best_model].fit(X, y)

In [53]:
drop_list = []
df = pd.read_csv("./Utah_Crash_Data_2020.csv")
df = df.drop([
"CRASH_ID",
# "CRASH_DATETIME",
"CRASH_SEVERITY_ID",
"ROUTE",
"MILEPOINT",
"LAT_UTM_Y",
"LONG_UTM_X",
"MAIN_ROAD_NAME",  
"CITY",                              
"COUNTY_NAME",                       
"WORK_ZONE_RELATED",                 
"PEDESTRIAN_INVOLVED",               
"BICYCLIST_INVOLVED",                
"MOTORCYCLE_INVOLVED",               
"IMPROPER_RESTRAINT",                
"UNRESTRAINED",                      
"DUI",                               
"INTERSECTION_RELATED",              
"WILD_ANIMAL_RELATED",               
"DOMESTIC_ANIMAL_RELATED",           
"OVERTURN_ROLLOVER",                 
"COMMERCIAL_MOTOR_VEH_INVOLVED",     
"TEENAGE_DRIVER_INVOLVED",           
"OLDER_DRIVER_INVOLVED",             
"NIGHT_DARK_CONDITION",              
"SINGLE_VEHICLE",                    
"DISTRACTED_DRIVING",                
"DROWSY_DRIVING",                   
"ROADWAY_DEPARTURE"
], axis=1)
df = df.dropna()

C:\Users\whyri\AppData\Local\Temp\ipykernel_8024\4135842229.py:2: DtypeWarning: Columns (0,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Utah_Crash_Data_2020.csv")


In [54]:
#Takes around 2:30 min to run
df["CRASH_MONTH"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.month
df["CRASH_WEEKDAY"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.day_name()
df["CRASH_HOUR"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.hour
df = df.drop(columns=["CRASH_DATETIME"])
df

,CRASH_MONTH,CRASH_WEEKDAY,CRASH_HOUR
0,2,Friday,10
1,12,Tuesday,12
2,12,Saturday,20
3,12,Saturday,0
4,12,Wednesday,17
...,...,...,...
252495,11,Thursday,23
252496,10,Tuesday,19
252497,10,Friday,17
252498,11,Wednesday,18


In [55]:
df_agg = df.groupby(['CRASH_MONTH', 'CRASH_WEEKDAY','CRASH_HOUR']).size().reset_index(name="COUNT")
#Although the below columns are numeric, they are categorical because they are a time not a measurement
df_agg = pd.get_dummies(df_agg, columns=["CRASH_MONTH","CRASH_WEEKDAY","CRASH_HOUR"])
df_agg.set_index("COUNT")

,CRASH_MONTH_1,CRASH_MONTH_2,CRASH_MONTH_3,CRASH_MONTH_4,CRASH_MONTH_5,CRASH_MONTH_6,CRASH_MONTH_7,CRASH_MONTH_8,CRASH_MONTH_9,CRASH_MONTH_10,...,CRASH_HOUR_14,CRASH_HOUR_15,CRASH_HOUR_16,CRASH_HOUR_17,CRASH_HOUR_18,CRASH_HOUR_19,CRASH_HOUR_20,CRASH_HOUR_21,CRASH_HOUR_22,CRASH_HOUR_23
COUNT,,,,,,,,,,,,,,,,,,,,,
54,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
model = fit_crossvalidate_reg(df_agg, 'COUNT', 5, 2)